***stock Evaluation***
********************************************************************************
**This script aims to evaluate, based on the hystorical date of an stock, how much is convenient to reduce the occurence of the investment in the stock**

***Libraries Definition***

In [1]:
##Generic library for Array and Data-time format
import datetime as dt
import numpy as np
import pandas as pd

##Generic library to create plots
import plotly.graph_objects as go
import plotly.subplots as sp

##Generic library to retrieve stock-Data
import yfinance as yf

    
##Github
my_token = ""

GENERAL PURPOSE FUNCTIONS
=========================

In [2]:
##Return the DATA INFORMATIONS
def month_year():
    now = dt.datetime.now()
    return now.day, now.month, now.year

In [3]:
import requests
from io import StringIO

def collect_data_from_github(github_raw_url):
    try:
        # L'URL del contenuto raw su GitHub (senza token nell'URL)
        # Esegui la richiesta GET
        response = requests.get(github_raw_url)
        response.raise_for_status()  # Solleva un errore se la risposta non è 200

        # Leggi il contenuto del file come CSV
        csvfile = StringIO(response.text)
        df = pd.read_csv(csvfile, sep=';', header=0)
        
        # Ritorna il DataFrame
        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

Here we have the function that retrieve the stock information based on the choosen ISIN

Generic Functions to retrieve Stock Data from Database
======================================================

In [4]:
##Return the yfinance.Ticker object that stores all the relevant stock informations
from pandas import DataFrame
import warnings

def get_stock_data(isin_string):
    try:
        stock_ticker = isin_string
        stock_data = yf.Ticker(stock_ticker)
        stock_info = stock_data.info  # Effettua una richiesta per ottenere le informazioni
        return stock_data
    except ValueError as e:
        print(f"Errore nel recuperare i dati per {isin_string}: {e}")
        return None
    except Exception as e:
        print(f"Si è verificato un errore non previsto per {isin_string}: {e}")
        return None



##Return the hystorical data with date expressed as string --> Suitable for calculations
def get_stock_with_date_index_data(
    stock_data, start_date, end_date, ma_period=200
) -> DataFrame | DataFrame:
    full_date_range = pd.date_range(start=start_date, end=end_date, freq="D")
    try:
        if isinstance(stock_data, yf.Ticker):
            hist_data = stock_data.history(start=start_date, end=end_date)
            hist_data.index = hist_data.index.strftime("%Y-%m-%d")
            hist_data = hist_data.reindex(full_date_range.strftime("%Y-%m-%d"))
            hist_data.infer_objects().ffill(inplace=True)

            hist_data_to_return = pd.DataFrame(
                index=full_date_range.strftime("%Y-%m-%d"),
                columns=["symbol","stock_price", "volume", "MA200", "OBV"],
            )
            hist_data_to_return["stock_price"] = hist_data["Close"]
            hist_data_to_return["volume"] = hist_data["Volume"]
            hist_data_to_return["symbol"] = stock_data.ticker
            hist_data_to_return["MA200"] = (
                hist_data_to_return["stock_price"].rolling(window=ma_period).mean()
            )

            # Calcolo dell'indicatore OBV
            hist_data_to_return["OBV"] = np.where(
                hist_data_to_return["stock_price"].diff() > 0,  # type: ignore
                hist_data_to_return["volume"],
                -hist_data_to_return["volume"],
            )
            with warnings.catch_warnings():
                warnings.simplefilter('ignore', FutureWarning)
                hist_data_to_return.ffill(inplace=True)
            return hist_data_to_return
        else:
            raise ValueError(f"stock_data must be a yf.Ticker object, got {type(stock_data)} instead.")
    except Exception as e:
        print(f"Error: {e}")
        hist_data_to_return = pd.DataFrame(
            index=full_date_range.strftime("%Y-%m-%d"),
            columns=["symbol","stock_price", "volume", "MA200", "OBV"],
        )
        return hist_data_to_return

Functions Related to Investment Strategies
=========================================

In [5]:
import warnings
def get_info_investment(
    stock_data, invested_capital, start_date, end_date, purchase_frequency
):
    # Crea un intervallo di date dal start_date al end_date con purchase_frequency interval
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', FutureWarning)
        purchase_dates = (
            pd.date_range(start_date, end_date, freq=purchase_frequency)
            ).strftime("%Y-%m-%d")
    # Crea un DataFrame vuoto che coprirà ogni giorno tra start_date e end_date
    daily_investment_df = pd.DataFrame(
        index=(pd.date_range(start=start_date, end=end_date, freq="D")).strftime(
            "%Y-%m-%d"
        ),
        columns=[
            "price",
            "shares_bought",
            "average_cost",
            "total_investment",
            "total_shares",
            "daily_stock_price",
            "purchase_dates",
        ],
    )
    ##To suppress some Pandas Warnings
    total_investment = 0
    total_shares = 0

    # Ciclo attraverso ogni data di acquisto
    for date in purchase_dates:
        purchase_amount = invested_capital / len(purchase_dates)
        total_investment += purchase_amount
        daily_stock_price = stock_data.loc[date, "stock_price"]
        shares_bought = purchase_amount / daily_stock_price if daily_stock_price else 0
        total_shares += shares_bought
        # Imposta i valori per il giorno di acquisto
        daily_investment_df.loc[date] = [
            daily_stock_price,
            shares_bought,
            total_investment / total_shares if total_shares else 0,
            total_investment,
            total_shares,
            daily_stock_price,
            purchase_dates,
        ]

    # Riempie in avanti i giorni senza acquisti con i valori dell'ultimo acquisto noto
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', FutureWarning)
        daily_investment_df.ffill(inplace=True)

    # Stock price between purchase_dates
    daily_investment_df["daily_stock_price"] = stock_data["stock_price"]

    # Calcola i valori di mercato giornalieri e i guadagni
    daily_investment_df["market_value"] = (
        daily_investment_df["daily_stock_price"] * daily_investment_df["total_shares"]
    )
    daily_investment_df["daily_gain"] = (
        daily_investment_df["market_value"] - daily_investment_df["total_investment"]
    )
    daily_investment_df["daily_gain_perc"] = (
        daily_investment_df["daily_gain"] / daily_investment_df["total_investment"]
    ) * 100

    # Riempe in NAN
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', FutureWarning)
        daily_investment_df.ffill(inplace=True)

    final_data = {
        "symbol": stock_data["symbol"],
        "average_cost": daily_investment_df["average_cost"],
        "market_value": daily_investment_df["market_value"],
        "daily_gain": daily_investment_df["daily_gain"],
        "daily_gain_perc": daily_investment_df["daily_gain_perc"],
        "total_shares": daily_investment_df["total_shares"],
        "total_investment": daily_investment_df["total_investment"],
        "purchase_dates": daily_investment_df["purchase_dates"].to_list(),
    }

    return final_data


# Find which is the best strategy of investment
def get_best_investment_strategy(results):
    best_strategy = None
    best_average_cost = float("inf")
    best_number_shares = 0
    best_return_value = 0
    best_market_value = 0

    for freq, result in results.items():
        average_cost = result["average_cost"].iloc[-1]
        number_shares = result["total_shares"].iloc[-1]
        last_date_purchase = result["purchase_dates"][-1]  ##It is a list
        final_return_value = result["daily_gain"].iloc[-1]
        market_value = result["market_value"].iloc[-1]

        if average_cost < best_average_cost:
            best_strategy = freq
            best_average_cost = average_cost
            best_number_shares = number_shares
            best_last_date_purchase = last_date_purchase
            best_return_value = final_return_value
            best_market_value = market_value
    print(
        f"The winning strategy is {best_strategy} with an average cost of {best_average_cost:.2f} , {best_number_shares:.2f} shares and last purchase on {best_last_date_purchase} with a return value of {best_return_value} USD."
    )
    print(f"The final market value at is : {best_market_value} USD")
    return (best_strategy, best_average_cost, best_number_shares, best_market_value)

GENERIC PLOT FUNCTIONS
===========================

In [6]:
##To be used in order to plot the stock behavior along two dates that you choose
def plot_stock_data(ticker, stock_data, start_date, end_date):
    try:
        ## hist_data = stock_data.history(start=start_date, end=end_date)
        fig = go.Figure(
            data=[go.Scatter(x=stock_data.index, y=stock_data["stock_price"])]
        )
        fig.update_layout(
            title=f"stock <b>{ticker}</b> Price History",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
        )
        fig.show()
    except Exception as e:
        print(f"Error: {e}")


def create_plot(x, y, name_trace, name_graph, xaxis_title, yaxis_title):
    fig = go.Figure()
    for x_list, y_list, name in zip(x, y, name_trace):
        fig.add_trace(
            go.Scatter(
                x=x_list,
                y=y_list,
                mode="lines+markers+text",
                name=name,
            )
        )
    fig.update_layout(
        title=name_graph,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        legend_title="Legenda",
        hovermode="x",
    )
    fig.show()
    return fig

***MAIN CODE***
===========================

Hystorical Behavior of the Selected stock
===========================

In [22]:
import datetime as dt


### GENERIC INFORMATION ABOUT stock and its HYSTORICAL BEHAVIOR
# Time Informations
day, month, year = month_year()
my_portfolio_analysis = int(
    input("Enter 0 for Single Stock Analysis or 1 for My Portfolio Analysis")
)
if my_portfolio_analysis:
    github_raw_url = "https://raw.githubusercontent.com/giuseppedavidde/Data_for_Analysis/main/My_Portfolio.csv"
    stock_under_test = collect_data_from_github(github_raw_url)
else:
    stock_under_test = input("Enter the stock ticker symbol: ")
    start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = f"{year}-{month}-{day}"
if isinstance(stock_under_test, pd.DataFrame):
    stock_data_object = {}
    for symbol in stock_under_test["Symbol"]:
        if pd.notna(symbol):
            stock_data_object[symbol] = get_stock_data(symbol)
else:
    stock_data_object = get_stock_data(stock_under_test)

Evaluation with DCA with Custom Purchase Frequency
===========================

User Input Data
===============

In [ ]:
from datetime import datetime as dtm

### Percentuale di peso sul portafoglio
### Ribilanciare in automatico (in una altra sezione!)
if isinstance(stock_under_test, pd.DataFrame):
    stock_data_dca_values = {}
    end_date_dca = f"{year}-{month}-{day}"
    ### Capitale Investito per ogni ETF/Stock/Crypto
    invested_capital_usd_dict = {}
    invested_capital_eur_dict = {}
    capital_by_category = {category: 0.0 for category in stock_under_test["Category"].unique()}
    weight_by_symbol = {}
    #############################
    # USDEUR DailyRate Change
    forex_data = stock_data_object["USDEUR=X"].history(period="1d")
    latest_rate = forex_data["Close"].iloc[-1]
    #############################    
    for symbol in stock_under_test["Symbol"]:
        category = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Category"].iloc[0]
        shares = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Shares"].iloc[0]
        invested_capital_usd_dict[symbol] = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Invested (USD)"].iloc[0]
        invested_capital_eur_dict[symbol] = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Invested (USD)"].iloc[0] * latest_rate if (stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Invested (USD)"].iloc[0] > 0.0) else stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Invested (EUR)"].iloc[0]
        if category not in capital_by_category:
            capital_by_category[category] = 0.0
        capital_by_category[category] += invested_capital_eur_dict[symbol]
        weight_by_symbol[symbol] = {"invested": invested_capital_eur_dict[symbol], "category": category}
        ####### STOCK DATA based on Last Purchase Date ######    
        stock_data_dca_values[symbol] = get_stock_with_date_index_data(stock_data_object[symbol], stock_under_test.loc[stock_under_test["Symbol"] == symbol,"Last Purchase (YY-MM-DD)"].iloc[0], end_date_dca, ma_period=200)
        #####################################################
    invested_capital_usd = stock_under_test["Total Invested (USD)"].iloc[0]
    invested_capital_eur = stock_under_test["Total Invested (EUR)"].iloc[0]
    invested_capital_eur += (invested_capital_usd*latest_rate) 
    #Bisogna avere invested_capital diviso per ogni stock/ETF per poter procedere con il get_info_investment
    invested_capital_invested = float(invested_capital_eur)
    print(f"invested_capital EUR : {invested_capital_invested}")
    # Calcolo delle percentuali di peso per simbolo
    for symbol, data in weight_by_symbol.items():
        category_total = capital_by_category[data["category"]]
        if category_total > 0.0:
            weight_by_symbol[symbol]["weight"] = data["invested"] / category_total * 100
        else:
            weight_by_symbol[symbol]["weight"] = 0
    # Stampa o utilizzo delle informazioni calcolate
    for category, total in capital_by_category.items():
        print(f"Total invested in {category}: €{total:.2f}")
    for symbol, data in weight_by_symbol.items():
        print(f"{symbol} has a weight of {data['weight']:.2f}% in the {data['category']} category")
else:
    invested_capital_invested = float(input("Insert how much you want to invest in EUR"))
    start_date_dca = input("Enter the start date for DCA Strategy (YYYY-MM-DD): ")
    end_date_dca = input("Enter the end date for DCA Strategy (YYYY-MM-DD): ")
    stock_data_dca_values = get_stock_with_date_index_data(
    stock_data_object, start_date_dca, end_date_dca, ma_period=200
    )

In [28]:
# Creazione del grafico a torta per il capitale totale investito per categoria
def create_pie_chart_for_category(capital_by_category,title_text):
    labels = list(capital_by_category.keys())
    values = list(capital_by_category.values())

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.3)])
    fig.update_layout(title_text=f"{title_text}")
    fig.show()

# Creazione del grafico a torta per il peso percentuale di ogni simbolo all'interno della sua categoria
def create_pie_chart_for_symbol_weight(weight_by_symbol):
    for category in capital_by_category.keys():
        category_symbols = [symbol for symbol, data in weight_by_symbol.items() if data['category'] == category]
        category_weights = [data['weight'] for symbol, data in weight_by_symbol.items() if data['category'] == category]

        fig = go.Figure(data=[go.Pie(labels=category_symbols, values=category_weights, hole=0.3)])
        fig.update_layout(title_text=f'Peso Percentuale per Simbolo in Categoria "{category}"')
        fig.show()

# Chiamata delle funzioni per visualizzare i grafici
create_pie_chart_for_category(capital_by_category,'Capitale Totale Investito per Categoria')
create_pie_chart_for_symbol_weight(weight_by_symbol)


Plot Stock/ETF with some statistics
=====================================

In [9]:
plot_stock_under_test = int(
    input("Enter 0 to skip Plot or 1 to Plot your Portfolio ETFs/Stocks")
)
if plot_stock_under_test:
    if isinstance(stock_data_dca_values, dict):
        for symbol, df in stock_data_dca_values.items():
            dates = df.index  # Prendi l'indice del DataFrame per il simbolo corrente
            
            # Prepara i dati per il plot
            plot_data = {
                "y": [df["stock_price"], df["MA200"], df["OBV"]],
                "x": [dates, dates, dates],  # Stesse date per tutte le serie
                "name_trace": [f"{symbol} Stock Price", f"{symbol} MA200", f"{symbol} OBV"],
                "name_graph": f"{symbol} Stock Price and Moving Average and OBV",
                "xaxis_title": "Date",
                "yaxis_title": "Price",
            }

            # Chiamata alla funzione 'create_plot'
            fig = create_plot(
                x=plot_data["x"],
                y=plot_data["y"],
                name_graph=plot_data["name_graph"],
                name_trace=plot_data["name_trace"],
                xaxis_title=plot_data["xaxis_title"],
                yaxis_title=plot_data["yaxis_title"],
                )
    else: 
        dates = stock_data_dca_values.index
        plot_data = {
            "y": [
                stock_data_dca_values["stock_price"],
                stock_data_dca_values["MA200"],
                stock_data_dca_values["OBV"],
            ],
            "x": [dates, dates, dates],  # Usa le stesse date per stock_price e MA200
            "name_trace": ["Stock Price", "MA200", "OBV"],  # Nomi delle serie
            "name_graph": f"{stock_under_test} Stock Price and Moving Average and OBV",
            "xaxis_title": "Date",
            "yaxis_title": "Price in USD",
        }
        # Chiamata alla funzione 'create_plot'
        fig = create_plot(
            x=plot_data["x"],
            y=plot_data["y"],
            name_graph=plot_data["name_graph"],
            name_trace=plot_data["name_trace"],
            xaxis_title=plot_data["xaxis_title"],
            yaxis_title=plot_data["yaxis_title"],
        )



Calculations
========================

In [10]:
# Supponendo che start_date_dca e end_date_dca siano gia' stati inseriti e siano nel formato corretto.
import pprint ##To print Dictionary in Python

end_date = dtm.strptime(end_date_dca, "%Y-%m-%d")

if isinstance(stock_data_dca_values, dict):
    delta_months = {}
    start_date = {}
    for symbol in stock_under_test["Symbol"]:
        start_date[symbol] = dtm.strptime(stock_under_test.loc[stock_under_test["Symbol"]==symbol,"Last Purchase (YY-MM-DD)"].iloc[0], "%Y-%m-%d")
        delta_months[symbol] = ((end_date.year - start_date[symbol].year) * 12 + end_date.month - start_date[symbol].month)
        # Se vuoi considerare anche i giorni per un conteggio più preciso (se il giorno di fine è dopo il giorno di inizio)
        if end_date.day > start_date[symbol].day: delta_months[symbol] += 1
        # print(f"The number of months for {symbol} between start_date {stock_under_test.loc[stock_under_test["Symbol"]==symbol,"Last Purchase (YY-MM-DD)"].iloc[0]} and end_date {end_date_dca} is {delta_months}.")
        purchase_frequencies = [
        "1M",
        "3M",
        "6M",
        "9M",
        "12M",
        "15M",
        f"{delta_months[symbol]}M",  ##SUM LUMP
        ]  # range of purchase frequencies   
else:
    start_date = dtm.strptime(start_date_dca, "%Y-%m-%d")
    # Calcolo della differenza in mesi per trovare il valore di mesi per effettuare un solo acquisto (LAMP SUM approach)
    delta_months = ((end_date.year - start_date.year) * 12 + end_date.month - start_date.month)    
    print(f"The number of months between {start_date_dca} and {end_date_dca} is {delta_months}.")
    purchase_frequencies = [
    "1M",
    "3M",
    "6M",
    "9M",
    "12M",
    "15M",
    f"{delta_months}M",  ##SUM LUMP
    ]  # range of purchase frequencies



results = {}
global_market_values = {freq: pd.Series(dtype=float) for freq in purchase_frequencies}

for freq in purchase_frequencies:
    if isinstance(stock_data_dca_values, dict):
        results[freq] = {}
        for symbol in stock_under_test["Symbol"]:
            usd_investment = 0  # Resetta la flag per ogni nuovo symbol
            start_date_symbol = stock_under_test.loc[stock_under_test["Symbol"] == symbol, "Last Purchase (YY-MM-DD)"].iloc[0]
            if invested_capital_usd_dict[symbol] > 0.0:
                result = get_info_investment(
                    stock_data_dca_values[symbol],
                    invested_capital_usd_dict[symbol],
                    start_date_symbol,
                    end_date_dca,
                    freq,
                )
                usd_investment = 1
            elif invested_capital_eur_dict[symbol] > 0.0:
                result = get_info_investment(
                    stock_data_dca_values[symbol],
                    invested_capital_eur_dict[symbol],
                    start_date_symbol,
                    end_date_dca,
                    freq,
                )
            results[freq][symbol] = result
            if usd_investment == 1 and "market_value" in result:
                results[freq][symbol]["market_value"] *= latest_rate

        # Aggiorna global_market_values dopo aver elaborato tutti i simboli per la frequenza corrente
        global_market_values[freq] = pd.Series(dtype=float)
        for symbol, result in results[freq].items():
            if "market_value" in result:
                if global_market_values[freq].empty:
                    global_market_values[freq] = result["market_value"]
                else:
                    global_market_values[freq] = global_market_values[freq].add(result["market_value"], fill_value=0)
    else:
        result = get_info_investment(
            stock_data_dca_values,
            invested_capital_invested,
            start_date_dca,
            end_date_dca,
            freq,
        )
        results[freq] = result



## pp = pprint.PrettyPrinter(indent=4)
## pp.pprint(results)
    #
# for freq, market_value_series in global_market_values.items():
#     print(f"Frequency: {freq}")
#     print(market_value_series)
#print(f"{results["1ME"]["GME"]}")

###### TROVARE IL VALORE DI INVESTIMENTO DA EFFETTUARE PER RIPORTARE A 0 LA MINUSVALENZA!
## num_azioni_attuali = 402.68
## costo_medio_attuale = 7.93
## prezzo_attuale = 1.02
## 
## investimento_aggiuntivo = (num_azioni_attuali * (costo_medio_attuale - prezzo_attuale)) / (1 - (1 / prezzo_attuale))
## 
## print(f"L'investimento aggiuntivo necessario è: {investimento_aggiuntivo:.2f} USD")


Plotting Section for DCA vs Sum Lump 
====================================

In [11]:
from dataclasses import field


plot_stock_under_test = int(
    input("Enter 0 to skip Plot or 1 to Plot your DCA vs Sum Lump ETFs/Stocks")
)
if plot_stock_under_test:
    import plotly.graph_objs as go
    from ipywidgets import interactive, HBox, VBox, widgets, Layout, ToggleButton
    # Definisci la funzione per creare e aggiornare il grafico
    # Inizializza un grafico vuoto che verrà aggiornato o sovrascritto
    global_fig = go.Figure()
    
    def create_interactive_plot(stock_data_df_or_dict,field,exception_field_string, exception_field_array, freq, new_plot):
            global global_fig
            if new_plot:  # Se l'utente ha scelto di creare un nuovo grafico
                global_fig = go.Figure()
            traces = []
            if isinstance(stock_data_df_or_dict, dict):
                if field == exception_field_string:
                    y_data = exception_field_array[freq]
                    x_data = exception_field_array[freq].index
                    traces.append(go.Scatter(
                            x=x_data, y=y_data, mode='lines', name=f"{freq} {field}"
                        ))
                else:
                    for symbol, stock_df in stock_data_df_or_dict.items():
                        if freq in results and symbol in results[freq]:
                            # Aggiungi il dato di questa stock alla lista 'y'
                            y_data = results[freq][symbol][field] 
                            x_data = results[freq][symbol][field].index
                            traces.append(go.Scatter(
                                x=x_data, y=y_data, mode='lines', name=f"{symbol} {freq} {field}"
                            ))
            else:
                # Aggiungi il dato di questa stock alla lista 'y'
                    y_data = results[freq][field]
                    traces.append(go.Scatter(
                    x=results[freq][field].index, y=y_data, mode='lines', name=f"{freq} {field}"
                    ))

                # Aggiorna o configura il grafico con le trace attuali
            for trace in traces:
                global_fig.add_trace(trace)

            global_fig.update_layout(
                title=f"{field.title()} over Time ({freq})",
                xaxis={'title': "Date"},
                yaxis={'title': field.title()},
                hovermode='closest'
                )

            global_fig.show()
            
     # Creare i widget per l'interattività
    if isinstance(stock_data_dca_values, dict):
        fields_to_plot = ["average_cost", "market_value", "daily_gain", "daily_gain_perc","global_market_values"]
    else:
        fields_to_plot = ["average_cost", "market_value", "daily_gain", "daily_gain_perc"]
        
    field_selector = widgets.Dropdown(options=fields_to_plot, value='market_value', description='Field')
    freq_selector = widgets.Dropdown(options=purchase_frequencies, value=purchase_frequencies[0], description='Frequency')
    new_plot_button = ToggleButton(description='New Plot', value=False)
    # Usa 'interactive' per collegare i widget alla funzione di creazione dei grafici
    interactive_plot = interactive(create_interactive_plot(stock_data_dca_values, field_selector, "global_market_values", global_market_values, freq_selector, new_plot_button), field=field_selector, freq=freq_selector,new_plot=new_plot_button)
    # Mostra i widget e il grafico interattivo
    display(HBox([field_selector, freq_selector, new_plot_button]))
    display(interactive_plot) 
  ##Define which is the best strategy
    # (best_strategy, best_average_cost, best_number_shares, best_market_value) = (
    #     get_best_investment_strategy(results)
    # )
